In [1]:
# (1) Download databricks.spark.xml library 
# (2) https://repo1.maven.org/maven2/com/databricks/spark-xml_2.12/0.12.0/spark-xml_2.12-0.12.0.jar
# (3) Upload jar file to HDFS root
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.jars", "hdfs://hdfs-nn:9000/spark-xml_2.12-0.12.0.jar") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
economy = spark.sql(
    """
    SELECT *
    FROM americancrimes.economy
    """
)
economy.toPandas()

,state,household_median_income,min_wage_state,min_wage_federal,cpi_average,unemployed,employed,employable_pop,gdp_state,year


In [3]:
politics = spark.sql(
    """
    SELECT *
    FROM americancrimes.politics
    """
)
politics.toPandas()

,state,party,candidatevotes,totalvotes,year


In [4]:
demography = spark.sql(
    """
    SELECT *
    FROM americancrimes.demography
    """
)
demography.toPandas()

,state_abb,state,total_pop,white,black,native,two_pop,asian_or_pacific_islander,year


In [5]:
crimes = spark.sql(
    """
    SELECT *
    FROM americancrimes.crimes
    """
)
crimes.toPandas()

,violent_crime,property_crime,state,state_abb,year


In [6]:
join1 = economy.join(politics, (politics.year == economy.year) & (politics.state == economy.state), "left_outer").select(economy.year, economy.state, economy.household_median_income, economy.min_wage_state, economy.gdp_state, economy.min_wage_federal, economy.cpi_average, economy.unemployed, economy.employed, economy.employable_pop, economy.gdp_state, politics.party, politics.candidatevotes, politics.totalvotes)
join1.toPandas()

,year,state,household_median_income,min_wage_state,gdp_state,min_wage_federal,cpi_average,unemployed,employed,employable_pop,gdp_state,party,candidatevotes,totalvotes


In [7]:
join2 = join1.join(crimes, (crimes.year == join1.year) & (crimes.state_abb == join1.state), "left_outer").select(join1.year, join1.state, join1.household_median_income, join1.min_wage_state, join1.gdp_state, join1.party, join1.candidatevotes, join1.totalvotes, crimes.violent_crime, crimes.property_crime)
join2.toPandas()

,year,state,household_median_income,min_wage_state,gdp_state,party,candidatevotes,totalvotes,violent_crime,property_crime


In [8]:
join3 = join2.join(demography, (demography.year == join2.year) & (demography.state_abb == join2.state), "left_outer").select(join2.year, join2.state, join2.household_median_income, join2.min_wage_state, join2.gdp_state, join2.party, join2.candidatevotes, join2.totalvotes, join2.violent_crime, join2.property_crime, demography.total_pop)
join3.toPandas()

,year,state,household_median_income,min_wage_state,gdp_state,party,candidatevotes,totalvotes,violent_crime,property_crime,total_pop


In [9]:
join3 = join3.fillna(0)
join3.toPandas()
join3.printSchema()

root
 |-- year: date (nullable = true)
 |-- state: string (nullable = true)
 |-- household_median_income: float (nullable = false)
 |-- min_wage_state: float (nullable = false)
 |-- gdp_state: float (nullable = false)
 |-- party: string (nullable = true)
 |-- candidatevotes: integer (nullable = true)
 |-- totalvotes: integer (nullable = true)
 |-- violent_crime: integer (nullable = true)
 |-- property_crime: integer (nullable = true)
 |-- total_pop: integer (nullable = true)



In [10]:
derivada1 = join3.withColumn("party_results_ratio", round((col("candidatevotes")/col("totalvotes"))*100, 2))
derivada2 = derivada1.withColumn("crimes_100k", (col("property_crime") + (col("violent_crime")))/col("total_pop")*100000) \
                     .withColumn("gdp_state", (col("gdp_state") * 1000000))
derivada2.toPandas()

,year,state,household_median_income,min_wage_state,gdp_state,party,candidatevotes,totalvotes,violent_crime,property_crime,total_pop,party_results_ratio,crimes_100k


In [11]:
us_state_abbrev = {
    'AL': 'US-AL',
    'AK': 'US-AK',
    'AZ': 'US-AZ',
    'AR': 'US-AR',
    'CA': 'US-CA',
    'CO': 'US-CO',
    'CT': 'US-CT',
    'DE': 'US-DE',
    'DC': 'US-DC',
    'FL': 'US-FL',
    'GA': 'US-GA',
    'HI': 'US-HI',
    'ID': 'US-ID',
    'IL': 'US-IL',
    'IN': 'US-IN',
    'IA': 'US-IA',
    'KS': 'US-KS',
    'KY': 'US-KY',
    'LA': 'US-LA',
    'ME': 'US-ME',
    'MD': 'US-MD',
    'MA': 'US-MA',
    'MI': 'US-MI',
    'MN': 'US-MN',
    'MS': 'US-MS',
    'MO': 'US-MO',
    'MT': 'US-MT',
    'NE': 'US-NE',
    'NV': 'US-NV',
    'NH': 'US-NH',
    'NJ': 'US-NJ',
    'NM': 'US-NM',
    'NY': 'US-NY',
    'NC': 'US-NC',
    'ND': 'US-ND',
    'OH': 'US-OH',
    'OK': 'US-OK',
    'OR': 'US-OR',
    'PA': 'US-PA',
    'RI': 'US-RI',
    'SC': 'US-SC',
    'SD': 'US-SD',
    'TN': 'US-TN',
    'TX': 'US-TX',
    'UT': 'US-UT',
    'VT': 'US-VT',
    'VA': 'US-VA',
    'WA': 'US-WA',
    'WV': 'US-WV',
    'WI': 'US-WI',
    'WY': 'US-WY'
}


abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

derivada2 = derivada2.replace(to_replace=us_state_abbrev, subset=['state'])

derivada2.toPandas()

,year,state,household_median_income,min_wage_state,gdp_state,party,candidatevotes,totalvotes,violent_crime,property_crime,total_pop,party_results_ratio,crimes_100k


In [12]:
derivada2.printSchema()

root
 |-- year: date (nullable = true)
 |-- state: string (nullable = true)
 |-- household_median_income: float (nullable = false)
 |-- min_wage_state: float (nullable = false)
 |-- gdp_state: float (nullable = false)
 |-- party: string (nullable = true)
 |-- candidatevotes: integer (nullable = true)
 |-- totalvotes: integer (nullable = true)
 |-- violent_crime: integer (nullable = true)
 |-- property_crime: integer (nullable = true)
 |-- total_pop: integer (nullable = true)
 |-- party_results_ratio: double (nullable = true)
 |-- crimes_100k: double (nullable = true)



In [13]:
# Reorder data
data =  derivada2.select(
        "state",
        "household_median_income",
        "min_wage_state",
        'gdp_state'
,'candidatevotes'
,'totalvotes'
,'violent_crime'
,'property_crime'
,'total_pop'
    ,'party'
,'party_results_ratio'
,'crimes_100k', "year")
data.toPandas()

,state,household_median_income,min_wage_state,gdp_state,candidatevotes,totalvotes,violent_crime,property_crime,total_pop,party,party_results_ratio,crimes_100k,year


In [14]:
data.repartition(1) \
    .write \
    .partitionBy("year") \
    .format("parquet") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/warehouse/americancrimes.db/derived_social_stats/")


In [15]:
spark.catalog.recoverPartitions("americancrimes.derived_social_stats")

spark.sql(
    """
    SELECT *
    FROM americancrimes.derived_social_stats
    """
).toPandas()

,state,household_median_income,min_wage_state,gdp_state,candidatevotes,totalvotes,violent_crime,property_crime,total_pop,party,party_results_ratio,crimes_100k,year
